# Fiber Assignment

This tutorial will teach you:
* The required inputs to fiber assignment.
* How to run `fiberassign` on those.
* How to interpret the fiber assignment output files, including understanding coverage.

This tutorial covers the high level `fiberassign` script.  See https://fiberassign.readthedocs.io/en/latest/ for installation instructions and descriptions of the lower level `fba_run`, `fba_merge`, `fba_run_qa` etc. scripts.

In this notebook the inputs to fiberassign are random targets. The notebooks `FiberAssignDECaLS.ipynb` and `FiberAssignMocks.ipnynb` show how 
to use inputs from observations and mock files, respectively.

See https://desi.lbl.gov/trac/wiki/Computing/JupyterAtNERSC for instructions on configuring jupyter kernels with pre-installed DESI software at NERSC.  This tutorial was last tested with the 22.2 kernel on April 29 2022

Stephen Bailey  
December 2019

## Basic python imports

In [ ]:
import os, sys, subprocess
from collections import Counter
from datetime import datetime

import numpy as np
from astropy.table import Table
from astropy.io import fits

import matplotlib.pyplot as plt

import desimodel.io
import desimodel.focalplane
import desimodel.footprint
from desitarget.targetmask import desi_mask, obsconditions

## Create an output directory in $SCRATCH/desi/test/fiberassign

In [ ]:
# You can set this to a local path if you have the software installed and a copy of the data.
# workdir = os.path.join(os.environ['HOME'], 'scratch', 'desi', 'tutorials', 'fiberassign_obs')
workdir = os.path.join(os.environ['SCRATCH'], 'desi', 'test', 'fiberassign')
os.makedirs(workdir, exist_ok=True)
os.chdir(workdir)
outdir = os.path.join(workdir, 'output')
os.makedirs(outdir, exist_ok=True)

# Fiber Assignment inputs

Fiber assignment requires the following input files:

**TODO**: document what each column is

**TODO**: some of these are no longer required in fiberassign 1.0.0; identify and remove those (likely BGS_TARGET, MWS_TARGET, and BRICKNAME).
    
* "Merged Target List" (MTL): FITS file with columns:
  * TARGETID
  * RA
  * DEC
  * DESI_TARGET
  * BGS_TARGET
  * MWS_TARGET
  * PRIORITY
  * SUBPRIORITY
  * BRICKNAME
  * OBSCONDITIONS
  * NUMOBS_MORE
* sky locations; FITS file with columns:
  * TARGETID
  * RA
  * DEC
  * DESI_TARGET
  * BGS_TARGET
  * MWS_TARGET
  * PRIORITY
  * SUBPRIORITY
  * BRICKNAME
  * OBSCONDITIONS

Standard stars can either be in the same file as the science targets, or they can
be in a separate file (with the same required columns).  `fiberassign` uses `DESI_TARGET`
mask bits `STD_FAINT`, `STD_BRIGHT`, and `STD_WD` to determine which targets may be
used as standard stars.

Optional inputs:

* footprint tiles (default `$DESIMODEL/data/footprint/desi-tiles.fits`) with columns
  * TILEID
  * RA
  * DEC
  * PASS
  * OBSCONDITIONS
  * IN_DESI
  * PROGRAM



# Fiber Assignment from scratch

First we'll run fiberassignment on a set of random targets and tiles to understand the required inputs.  Then we'll proceed with running it on real data for real DESI tile locations.

## Custom tiling

We'll start by generting a set of offset tiles

In [ ]:
ntiles = 5
tiles = Table()
tile_radius = desimodel.focalplane.get_tile_radius_deg()
tiles['TILEID'] = np.arange(ntiles, dtype='i4')
tiles['RA'] = 2 + np.arange(ntiles)*0.5*tile_radius
tiles['DEC'] = np.zeros(ntiles)
tiles['PASS'] = np.zeros(ntiles, dtype='i2')
tiles['OBSCONDITIONS'] = np.ones(ntiles, dtype='i4') * obsconditions.DARK
tiles['IN_DESI'] = np.ones(ntiles, dtype='i2')
tiles['PROGRAM'] = np.full(ntiles, 'DARK', dtype='S6')
tiles.write('tiles.fits', format='fits', overwrite=True)

In [ ]:
def plot_sky_circle(x, y, radius, alpha=0.2):
    theta = np.linspace(0, 2*np.pi, 50)
    xx = x + np.cos(np.radians(y))*radius*np.cos(theta)
    yy = y + radius*np.sin(theta)
    plt.fill(xx, yy, alpha=alpha)

def plot_tile(ra, dec, alpha=0.2):
    tile_radius = desimodel.focalplane.get_tile_radius_deg()
    plot_sky_circle(ra, dec, tile_radius)

def plot_tiles(tiles):
    for i in range(len(tiles)):
        plot_tile(tiles['RA'][i], tiles['DEC'][i])
    plt.xlabel('RA [degrees]')
    plt.ylabel('dec [degrees]')

plt.figure(figsize=(8,4))
plot_tiles(tiles)

Let's compare that tiling coverage to the fiber coverage.  We'll see that the coverage of
actual fibers (positioners) is less than that of the tile, and has a more complex spatial structure.  By default, desimodel returns the realistic state of the focalplane at the time you run this notebook.  That is great for working in realtime on real data.  For this exercise, we will be using a focalplane from a time prior to commissioning so that we will have all positioners working and with their nominal properties.

In [ ]:
rundatestr = "2022-01-01T00:00:00+00:00"
rundate = datetime.strptime(rundatestr, "%Y-%m-%dT%H:%M:%S+00:00")
print(rundate)

In [ ]:
def plot_positioners(tilera, tiledec, alpha=0.1):
    fp, exclude, state, tmstr = desimodel.io.load_focalplane(time=rundate)
    # Select just science positioners
    rows = np.where(fp['DEVICE_TYPE'] == 'POS')[0]
    fp = fp[rows]
    ra, dec = desimodel.focalplane.xy2radec(tilera, tiledec, fp['OFFSET_X'], fp['OFFSET_Y'])
    plt.plot(ra, dec, '.', alpha=alpha)

plt.figure(figsize=(8,4))
for i in range(len(tiles)):
    plot_positioners(tiles['RA'][i], tiles['DEC'][i])

plt.xlabel('RA [degrees]')
plt.ylabel('dec [degrees]')

## Random targets

Now we'll generate randomly distributed targets.  To facilitate counting, we'll trim those
to just targets that are potentially covered by a tile (but might still not be covered by
any fibers)

In [ ]:
def generate_random_targets(density, tiles):
    #- Get basic bounds; don't worry about RA wraparound for this example
    tile_radius = desimodel.focalplane.get_tile_radius_deg()
    ramin = np.min(tiles['RA'] - tile_radius*np.cos(np.radians(tiles['DEC'])))
    ramax = np.max(tiles['RA'] + tile_radius*np.cos(np.radians(tiles['DEC'])))
    decmin = np.min(tiles['DEC']) - tile_radius
    decmax = np.max(tiles['DEC']) + tile_radius
    
    area = (ramax-ramin) * np.degrees((np.sin(np.radians(decmax)) - np.sin(np.radians(decmin))))
    n = int(area*density)

    #- Iterate if needed to get unique TARGETIDs
    while True:
        targetids = np.random.randint(0, 2**62-1, n)
        if len(set(targetids)) == n:
            break

    #- Create targets table
    targets = Table()
    targets['TARGETID'] = targetids
    targets['RA'] = np.random.uniform(ramin, ramax, n)
    phimin = np.radians(90-decmin)
    phimax = np.radians(90-decmax)
    targets['DEC'] = 90-np.degrees(np.arccos(np.random.uniform(np.cos(phimin), np.cos(phimax), n)))
    targets['DESI_TARGET'] = np.zeros(n, dtype='i8')
    targets['BGS_TARGET'] = np.zeros(n, dtype='i8')
    targets['MWS_TARGET'] = np.zeros(n, dtype='i8')
    targets['SUBPRIORITY'] = np.random.uniform(0, 1, n)
    targets['BRICKNAME'] = np.full(n, '000p0000')    #- required !?!
    targets['BRICKID'] = np.full(n, 0)    #- required !?!
    targets['BRICK_OBJID'] = np.arange(n)
    
    #- dummy values for fluxes
    for filt in ['G', 'R', 'Z']:
        targets['FIBERFLUX_'+filt] = np.zeros(n, dtype='f4')
        targets['FIBERFLUX_IVAR_'+filt] = np.ones(n, dtype='f4')
    
    #- Trim to targets that are covered by a tile
    ii = desimodel.footprint.is_point_in_desi(tiles, targets['RA'], targets['DEC'])
    targets = targets[ii]
    
    return targets


In [ ]:
#- ELG-like sample: high density, only one requested observation per target
density = 2400
targets = generate_random_targets(density, tiles)
n = len(targets)
targets['PRIORITY'] = 1000
targets['SUBPRIORITY'] = np.random.uniform(0, 1, n)
targets['DESI_TARGET'] = desi_mask.ELG
targets['OBSCONDITIONS'] = np.ones(n, dtype='i4') * obsconditions.DARK
targets['NUMOBS_MORE'] = np.ones(n, dtype='i8')
targets.meta['EXTNAME'] = 'MTL'
targets.write('mtl.fits', overwrite=True)

In [ ]:
#- Sky targets at 4x the density of fibers
fiber_density = 5000 / 7.5
sky_density = 4*fiber_density
sky = generate_random_targets(sky_density, tiles)
nsky = len(sky)
sky['DESI_TARGET'] = desi_mask.SKY
sky['OBSCONDITIONS'] = np.ones(nsky, dtype='i4') * obsconditions.mask('DARK|GRAY|BRIGHT')
sky.meta['EXTNAME'] = 'SKY'
sky.write('sky.fits', overwrite=True)

In [ ]:
std_density = 50
stdstars = generate_random_targets(std_density, tiles)
nstd = len(stdstars)
stdstars['SUBPRIORITY'] = np.random.uniform(0, 1, nstd)
stdstars['DESI_TARGET'] = desi_mask.STD_FAINT
stdstars['OBSCONDITIONS'] = np.ones(nstd, dtype='i4') * obsconditions.DARK
stdstars.meta['EXTNAME'] = 'STD'
stdstars.write('stdstars.fits', overwrite=True)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(targets['RA'], targets['DEC'], 'k,', alpha=0.3, label='targets')
plt.plot(stdstars['RA'], stdstars['DEC'], 'rx', alpha=0.5, label='stdstars')
plot_tiles(tiles)
plt.legend()

## Run fiberassign on those targets/standards/sky

Fiberassign calls desimodel to get focalplane properties.  By default this uses the current time when you run this notebook.  Instead, we will use the same historical time above in order to get a nominal focalplane prior to commissioning will all positioners working.

In [ ]:
cmd = 'fiberassign --overwrite --mtl mtl.fits --stdstar stdstars.fits --sky sky.fits'
cmd += ' --rundate {}'.format(rundatestr)
cmd += ' --footprint ./tiles.fits'
cmd += ' --outdir ./output/'
cmd = cmd.format(outdir=outdir)

print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())


## fiberassign output

`fiberassign` output files contain the following HDUs:
* `FIBERASSIGN`: fiber assignments for the 5000 positioners with fibers connected to a spectrograph
* `POTENTIAL_ASSIGNMENTS`: a table of what targets were covered by what fibers, whether or not they were assigned
* `SKY_MONITOR`: assignments of the 20 fibers that go to the sky monitor camera (not the spectrographs)
* `TARGETS`: (deprecated) a copy of the input targets table with all columns

The latest version of fiberassign also has HDUS `FASSIGN`, `FTARGETS`, and `FAVAIL`, which we won't cover here (yet).

Note: previously the primary fiberassign output files were called `tile-*.fits`, which caused confusion
with the "tile file" defining the DESI footprint, so these have been renamed `fiberassign-*.fits`.


In [ ]:
fafile = '{}/fiberassign-{:06d}.fits'.format(outdir, tiles['TILEID'][0])
fx = fits.open(fafile)
print(fx.info())
print('\nFIBERASSIGN columns:')
print(fx['FIBERASSIGN'].data.dtype.names)
print('\nPOTENTIAL_ASSIGNMENTS columns:')
print(fx['POTENTIAL_ASSIGNMENTS'].data.dtype.names)
fx.close()

## Which targets were assigned?

In [ ]:
assignments = list()
for tileid in tiles['TILEID']:
    fafile = '{}/fiberassign-{:06d}.fits'.format(outdir, tileid)
    assignments.append(Table.read(fafile, 'FIBERASSIGN'))

In [ ]:
assigned_targetids = np.concatenate([tmp['TARGETID'] for tmp in assignments])
isAssigned = np.isin(targets['TARGETID'], assigned_targetids)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(targets['RA'][isAssigned], targets['DEC'][isAssigned], 'k,')
plt.title('Targets assigned to fibers')
plot_tiles(tiles)

## And which targets weren't assigned?

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(targets['RA'][~isAssigned], targets['DEC'][~isAssigned], 'k,')
plt.title('Targets not assigned to fibers')

Let's compare the number that were assigned to the simple estimate of counting fibers.
For this density of targets, nearly all fibers are assigned to a target.
i.e. if a target wasn't assigned, it was because either it wasn't covered by a fiber at all,
or because that fiber was given to a different target instead.  Good.

In [ ]:
num_assigned = np.count_nonzero(isAssigned)
max_possible_assigned = int(len(tiles) * 5000 * 0.9)  #- 10% of fibers are reserved for std and sky calibrators
print('Targets assigned        ', num_assigned)
print('Max possible assignments', max_possible_assigned)
print('Ratio                   ', num_assigned/max_possible_assigned)

# Assigning a target more than once - part I

Now let's make a sample like Survey Validation, with a high target density but requesting 4 assignments per target when possible (`NUMOBS_MORE = 4`)

In [ ]:
#- Science Verification sample: high density, with multiple requested observations per target
density = 2400
targets = generate_random_targets(density, tiles)
n = len(targets)
targets['PRIORITY'] = 1000
targets['SUBPRIORITY'] = np.random.uniform(0, 1, n)
targets['DESI_TARGET'] = desi_mask.ELG
targets['OBSCONDITIONS'] = np.ones(n, dtype='i4') * obsconditions.DARK
targets['NUMOBS_MORE'] = np.ones(n, dtype='i8') * 4  #- THIS IS WHAT IS DIFFERENT THAN BEFORE
targets.meta['EXTNAME'] = 'MTL'
targets.write('mtl.fits', overwrite=True)

In [ ]:
#- Rerun fiberassign
#- Rerun fiberassign
print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())

### Histogram how many times each target was assigned

In [ ]:
def get_assigned_covered(tiles):
    assigned_ids = list()
    covered_ids = list()
    for tileid in tiles['TILEID']:
        fafile = '{}/fiberassign-{:06d}.fits'.format(outdir, tileid)
        assignments = Table.read(fafile, 'FIBERASSIGN')
        assigned_ids.extend(assignments['TARGETID'])

        #- Note: a single target could be covered by more than one fiber on the same
        #- tile, but it still can only be assigned once, so use np.unique() per tile
        #- to not double count those cases
        potential = Table.read(fafile, 'POTENTIAL_ASSIGNMENTS')
        covered_ids.extend(np.unique(potential['TARGETID']))

    #- Trim to just science targets (i.e. not SKY, not non-science STDSTAR)
    assigned_ids = np.array(assigned_ids)
    ii = np.in1d(assigned_ids, targets['TARGETID'])
    assigned_ids = assigned_ids[ii]

    covered_ids = np.array(covered_ids)
    ii = np.in1d(covered_ids, targets['TARGETID'])
    covered_ids = covered_ids[ii]

    return assigned_ids, covered_ids

#- Do a bit of counting magic to include targets that were never assigned or covered
def count_coverage(ids):
    c = Counter(ids)
    c.update(targets['TARGETID'])
    count = np.array(list(c.values())) - 1
    return count

In [ ]:
assigned_ids, covered_ids = get_assigned_covered(tiles)
assigned_count = count_coverage(assigned_ids)
covered_count = count_coverage(covered_ids)

In [ ]:
plt.subplot(211)
plt.hist(assigned_count, 11, (-0.5, 10.5), rwidth=0.8)
plt.xticks(np.arange(0,11))
plt.xlabel('number of times assigned')

plt.subplot(212)
plt.hist(covered_count, 11, (-0.5, 10.5), rwidth=0.8)
plt.xticks(np.arange(0,11))
plt.xlabel('number of times covered')

plt.tight_layout()

Why are the number of assignments so much less than the coverage?
The current fiberassign algorithm prioritizes observing more targets before
getting more observations of a previously observed target.  This is what we
want for Lyman-alpha QSOs during the main survey, but it isn't what we want
for Survey Validation (or LRGs).  GitHub [fiberassign ticket #140](https://github.com/desihub/fiberassign/issues/140) is tracking this.

**TODO** Add an option to fiberassign to give the flexibility to finish targets
before starting new ones

# Assigning a target more than once - part II

Let's repeat the `NUMOBS_MORE>1` exercise with a low density of science targets so that
they don't compete with each other so much.

In [ ]:
#- QSO-like sample: low density, multiple observations per target
density = 50  #- 
targets = generate_random_targets(density, tiles)
n = len(targets)
targets['PRIORITY'] = 1000
targets['SUBPRIORITY'] = np.random.uniform(0, 1, n)
targets['DESI_TARGET'] = desi_mask.ELG
targets['OBSCONDITIONS'] = np.ones(n, dtype='i4') * obsconditions.DARK
targets['NUMOBS_MORE'] = np.ones(n, dtype='i8') * 4  #- THIS IS WHAT IS DIFFERENT THAN BEFORE
targets.meta['EXTNAME'] = 'MTL'
targets.write('mtl.fits', overwrite=True)

In [ ]:
#- Rerun fiberassign
print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())

In [ ]:
assigned_ids, covered_ids = get_assigned_covered(tiles)
assigned_count = count_coverage(assigned_ids)
covered_count = count_coverage(covered_ids)

In [ ]:
plt.subplot(211)
plt.hist(assigned_count, 11, (-0.5, 10.5), rwidth=0.8)
plt.xticks(np.arange(0,11))
plt.xlabel('number of times assigned')

plt.subplot(212)
plt.hist(covered_count, 11, (-0.5, 10.5), rwidth=0.8)
plt.xticks(np.arange(0,11))
plt.xlabel('number of times covered')

plt.tight_layout()

That looks much better.  Let's confirm that the targets with low assignments are the ones with low coverage.

In [ ]:
n2d = np.zeros((10, 10), dtype=int)

count_assigned = Counter(assigned_ids)
count_covered = Counter(covered_ids)

for tid in targets['TARGETID']:
    n = count_assigned[tid]
    m = count_covered[tid]
    n2d[n,m] += 1

In [ ]:
plt.imshow(n2d)
plt.xlabel('Number of times Covered')
plt.ylabel('Number of times Assigned')

Indeed, nearly all cases of low assignments are due to low coverage

# Exercises

1. Run an intermediate density case where the number of targets times the number of requested observations is approximately equal to the total number of fibers.
2. Track down a case where a fiber received fewer than its maximum number of possible assignments based upon coverage.  Determine if the "missing" case was legitimate (e.g. if the fiber was assigned to a higher SUBPRIORITY target instead).
3. Generate an input target list with two different kinds of targets at the same density but different `PRIORITY`.  Run `fiberassign` on those and study the outputs.  Do the higher priority targets get more assigned fibers?

# TODO
* Add examples for running on real data, including using `desitarget.mtl.make_mtl` to convert targets -> mtl.
* When running fiberassign, capture stderr/stdout to a log and print an informative error message
  if it crashes.
* Add tutorials about lower-level `fba_run` and `fba_merge` commands.